## Relevant data is key. Right data types. Necessary columns. No missing data. And any other structural preferences

### before we do any analysis we have to load and prepare our dataset(s). In this case our desired dataset requires combining variables from more than 2 datasets

In [ ]:
#importing data manipulation library
import pandas as pd
import numpy as np

In [ ]:
#loading our data
data = 'raw_data/his_master_list_with_coordinates.xlsx'

site_data = pd.read_excel(data)

#sample 2 records for our new dataframe
site_data.sample(2)

In [ ]:
#check for duplicate data
site_data[site_data.duplicated()]

In [ ]:
#quick look at the columns to see the relevant ones
site_data.columns

In [ ]:
#removing the columns we do not need from the dataframe
site_data = site_data.drop(['CT', 'HTS', 'IL', 'MHealth', 'KP', 'EMR', 'EMRStatus',
       'DateOfEmrImplementation', 'ModeOfEMRUse', 'Lab_Manifest', 'MNCH', 'TB',
       'OVC', 'OTZ', 'PREP', 'KP.1', 'HIV', 'TPT', 'Covid_19', 'EVMMC',
       'HTS_Status', 'HTS_Use', 'HTS_Deployment', 'IL_Status', 'IL_WebADT',
       'IL_Mlab', 'IL_Ushauri', 'IL_lab_manifest', 'IL_3_PM', 'IL_AIR',
       'IL_nimeconfirm', 'MHealth_Nishauri', 'MHealth_C4C', 'MHealth_Ushauri',
       'MHealth_Mlab', 'MHealth_ART_Directory', 'MHealth_Psurvey'], axis=1)

In [ ]:
#since we are only interested in a specific implementing partner
site_data['SDP'].unique()

In [ ]:
#the implementing sdp to filter
partner_filter = 'KCCB ACTS'

#@allows us to pass our string variable through the query method. we also use method chaining to do addition manipulations on the resulting data.
kccb_sites = site_data.query('SDP == @partner_filter').reset_index().drop(['index'], axis=1)
kccb_sites.head(4)

In [ ]:
#checking for any missing values
kccb_sites.isna().value_counts()

In [ ]:
#saving our dataframe in a file for future use
kccb_sites.to_excel('processed_data/cleaned_kccb_his_master_coordinates.xlsx', index=False)

no data is missing and we can also see we have 105 records equating to __105 facility under kccb-acts sdp__

### our first dataset is cleaned and now we can do some few EDA on it

In [ ]:
#we can also use the value_counts() method to see the owners of the 105 sites
kccb_sites.Owner.value_counts()

64 sites belong to *__Kenya Episcopal Conference-Catholic Secretariat__*

### some quick visualization test

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(f'KCCB ACTS supports facilities in {kccb_sites.County.nunique()} counties')

In [ ]:
csites = kccb_sites.County.value_counts().reset_index()
csites.columns = ('county','Supported_Sites')
#csites.set_index('county')
#for index,county in csites.county.items():
#    print(str(county));
#basis on our plot; type and size
csites.plot.bar(figsize=(20,5))

#labels of plot axes using the import plt for matplotlib.pyplot
plt.xlabel('County')
plt.ylabel('No. of Supported Sites')
plt.title('Total Sites Supported by KCCB-ACTS as at August 2023', fontsize=13)

#for the xticks to show county names
plt.xticks(csites.index, labels=csites.county)

#plt.axis("off") #removes the borders and whitespace

#annotate values above bars based on index
for county, counts in csites.Supported_Sites.items():
    plt.text(county, counts+0.22, str(counts), fontsize=10, horizontalalignment='center');

### adding more variables to our data by merging with another dataset

In [ ]:
#loading dataset with the tx current data for the quarters
txdata = 'raw_data/kccbtxcurrdata.xlsx'

kccbtxdata = pd.read_excel(txdata, usecols={'mfl',
                                            'facility_name', 'txnew2023Q1','txnew2023Q2',
                                            'txnew2023Q3','txnew2023Q4',
                                            '2023Q1','2023Q2','2023Q3','2023Q4'}
                          ).sort_values(by='mfl',
                                        ascending=True)
kccbtxdata.tail(5)

In [ ]:
#to check on our dtypes
kccbtxdata.info()

In [ ]:
#show the last 5 records
kccb_sites.tail(5)

In [ ]:
#check for duplicate records
kccbtxdata[kccbtxdata.duplicated()]

In [ ]:
#check for missing values
kccbtxdata.isna().value_counts()

In [ ]:
#renaming the mfl column in the second df
kccbtxdata.rename(columns={'mfl':'mfl_code'}, inplace=True)

#merging the two dataframe where mfl_code is same
#and saving the resulting dataframe as a new dataframe to merge with the next dataset. use to_excel('dir/filename') to save it
tx_coords = (pd.
 merge(kccbtxdata, kccb_sites,
      on=['mfl_code'],
      how='left'))

### adding a dataset with 'region' variable for the sites

In [ ]:
#add regions
siteregions = 'raw_data/regiondata.csv'

region_data = pd.read_csv(siteregions, usecols=('Facility',
                                             'MFL Code',
                                             'Region'))

columns=('facility_name',
         'mfl_code',
         'region')

region_data.columns = columns

region_data.sample(1)

In [ ]:
region_data.shape

In [ ]:
#dataset only contains 57 sites, while our kccb_sites dataframe has 105 sites. 
#a quick check on null values in a merge of the two shows this
kccbsites_df = (pd.
 merge(tx_coords, region_data,
      on=['mfl_code'],
      how='left'))

#.fillna(value='Nairobi')
#.isnull().value_counts()

In [ ]:
kccbsites_df.sample(3)

In [ ]:
kccbsites_df.region.unique()

In [ ]:
#changing region from nan to the appropriate value for the facilities in the newer counties
#kccbsites_df[kccbsites_df.County.str.contains("Nairobi|Narok|Nyeri|Kirinyaga|Murang'a|Kiambu|Nakuru|Kajiado")].assign(region = "Nairobi") #worked but didnt change in place


In [ ]:
#rows containing the specific string as a filter
nairobi_region = kccbsites_df.County.str.contains("Nairobi|Narok|Nyeri|Kirinyaga|Murang'a|Kiambu|Nakuru|Kajiado")
mombasa_region = kccbsites_df.County.str.contains("Taita Taveta|Mombasa|Kilifi")

#use loc to take our filters in to fill in the nan values in regions
kccbsites_df.loc[nairobi_region, 'region'] = kccbsites_df.loc[nairobi_region, 'region'].fillna(value='Nairobi')
kccbsites_df.loc[mombasa_region, 'region'] = kccbsites_df.loc[mombasa_region, 'region'].fillna(value='Mombasa')

In [ ]:
kccbsites_df.shape

In [ ]:
kccbsites_df.sample(2)

In [ ]:
#check for any missing values
kccbsites_df.region.isna().value_counts()

In [ ]:
#our columns and their respective data types
kccbsites_df.dtypes

In [ ]:
#mfl_code to str since no mathematical operations should be done on it
kccbsites_df.mfl_code = kccbsites_df.mfl_code.apply(str)

In [ ]:
kccbsites_df = kccbsites_df.drop(['FacilityName','facility_name_y'], axis=1)

In [ ]:
#checking to see if our assigning of region is correct
filter = 'Nairobi'

(kccbsites_df
 .query('region==@filter')['County'].unique())

In [ ]:
kccbsites_df.rename(columns={'facility_name_x':'facility_name',
                        'County':'county',
                        'SubCounty':'sub_county',
                        'Owner':'owner'}, inplace=True)

# our final dataframe

In [ ]:
#isolate the relevant columns only
#save to excel file
prepd_data = (kccbsites_df[['mfl_code', 'facility_name', 'region',
       'county', 'sub_county', 'owner', 'lat', 'lon', 'txnew2023Q1', 'txnew2023Q2',
       'txnew2023Q3', 'txnew2023Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4']])


prepd_data.to_excel('processed_data/cleaned_data.xlsx', index=False)

In [ ]:
#prepd_data.memory_usage(deep=True).sum()

In [ ]:
#some light feature engineering, using np.sum and assign to make a new column to sums the columns of txnew for the quarter
#axis=0 would sum the rows
prepd_data.assign(txnew_total = np.sum(prepd_data[['txnew2023Q1', 'txnew2023Q2','txnew2023Q3', 'txnew2023Q4']], axis=1))

In [ ]:
print(f"Tx New Total for the sites by 2023Q4 is: {np.sum(prepd_data[['txnew2023Q1', 'txnew2023Q2','txnew2023Q3', 'txnew2023Q4']], axis=1).sum()}")

In [ ]:
#txq4 total by region
prepd_data.groupby(by=['region'])['2023Q4'].sum().reset_index().plot.barh()

In [ ]:
prepd_data[['region','2023Q4']].plot.scatter(x='region', y='2023Q4',figsize=(4,10))
plt.title("TX current distribution across Counties")
plt.xticks(rotation=90)

In [ ]:
prepd_data['2023Q4'].nlargest(10)

#### our streamlit app will mainly be using altair charts

In [ ]:
import altair as alt

In [ ]:
#the general flow of an altair chart
source = pd.DataFrame({
    "a": ["A", "B", "C"],
    "b": [28, 55, 43]
})

bar = alt.Chart(source).mark_bar().encode(
    y="a:N",
    x=alt.X("b:Q").scale(domain=[0, 60])
)
text = bar.mark_text(
    align="left",
    baseline="middle",
    dx=3
).encode(text="b")

bar + text

In [ ]:
#with our data 
data = prepd_data.groupby(by=['region'])['2023Q4'].sum().reset_index()

bar = alt.Chart(data).mark_bar().encode(
    y="region:N",
    x=alt.X("2023Q4:Q"))

text = bar.mark_text(
    align="left",
    baseline="middle",
    dx=3
).encode(text="2023Q4")

bar + text

In [ ]:
#with our data 

sitescount = kccb_sites.County.value_counts().reset_index()


bar = alt.Chart(sitescount).mark_bar().encode(
    y=alt.Y("County:N").title("").axis(labels=False),
    x=alt.X("count:Q").title("").axis(labels=False),
    color=alt.Color("County").scale(scheme="category20c"))

bar.configure_legend(
         padding=6,
         orient='bottom')

text = bar.mark_text(
    align="left",
    baseline="middle",
    dx=3
).encode(text="count")

bar + text

In [ ]:
quarter_tx = prepd_data.groupby(by=['region'])[['2023Q1','2023Q2','2023Q3','2023Q4']].sum().reset_index()


In [ ]:
quarter_tx.transpose().loc['2023Q1'].plot.bar()

In [ ]:
base = alt.Chart(data).mark_arc(innerRadius=50).encode(
    theta=alt.Theta(field="2023Q4", type="quantitative"),
    color=alt.Color(field="region", type="nominal"),
)

pie = base.mark_arc(outerRadius=150)
text = base.mark_text(radius=170, size=12).encode(
    text="2023Q4:N"
)

base + text

In [ ]:
prepd_data['2023Q4'].sum().dtype

In [ ]:
prepd_data.columns

In [ ]:

source = (prepd_data.groupby(
         by=['county'])[['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4']]
                  .sum())

alt.Chart(source).mark_rect().encode(
    alt.X("txnew2023Q4:Q").axis(labelAngle=0).title(""),
    alt.Y("county:O").title("Q4"),
    alt.Color("max(txnew2023Q4):Q").title(""),
)

In [ ]:
(prepd_data['2023Q3'] - prepd_data['2023Q2']).sum()

In [ ]:
(prepd_data.groupby(
         by=['county'])[['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4']]
                  .sum()
                  .reset_index())


In [ ]:
((prepd_data['txnew2023Q4']) - (prepd_data['txnew2023Q3'])).abs().sum()

In [ ]:
prepd_data[['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4']].sum()

In [ ]:
(prepd_data.groupby(
         by=['region'])[['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4']]
                  .sum()
                  .reset_index().transpose()).style.highlight_max()#.set_index(['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4'])


In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(prepd_data.groupby(by=['facility_name'])[['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4']].sum())

In [ ]:
import streamlit as st

In [ ]:
st.altair_chart(prepd_data.groupby(by=['region'])[['txnew2023Q1','txnew2023Q2','txnew2023Q3','txnew2023Q4']].sum().T)

seems i need a new dataset, showing county tested and new tx

In [ ]:
tested_data = 'raw_data/tested_totals.xlsx'
tested_df = pd.read_excel(tested_data)

In [ ]:
test_yield = alt.Chart((prepd_data.groupby(by=['county'])[['txnew2023Q4']].sum()).merge(tested_df,
                                                                 on=['county'],
                                                                 how='outer').set_index('county').reset_index()).mark_line().encode(
        x= alt.X("county:N"),
        y= alt.Y("number_tested:Q")).properties(
    height=500,
    width=1000)
                                                                 
txnewq = alt.Chart((prepd_data.groupby(by=['county'])[['txnew2023Q4']].sum()).merge(tested_df,
                                                                 on=['county'],
                                                                 how='outer').set_index('county').reset_index()).mark_area().encode(
        x= alt.X("county:N"),
        y= alt.Y("txnew2023Q4")).properties(
    height=500,
    width=1000)

txnewq + test_yield

In [ ]:
test = (prepd_data.groupby(by=['county'])[['txnew2023Q4']].sum()).merge(tested_df,
                                                                 on=['county'],
                                                                 how='outer').set_index("county").transpose()


In [ ]:
test.set_index()